In [3]:
import tokenizers
import transformers
from transformers import BertTokenizer, BertForMaskedLM
import sklearn
from sklearn.metrics import classification_report
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler
from transformers.pipelines import pipeline
from sklearn.linear_model import Ridge
import numpy as np
import pandas as pd
import torch
from tqdm import tqdm


In [4]:
train_df = pd.read_csv("C:/Users/bergo/OneDrive - University of Pisa/Tesi Magistrale/probing_data/probing_train.csv")
val_df = pd.read_csv("C:/Users/bergo/OneDrive - University of Pisa/Tesi Magistrale/probing_data/probing_test.csv")

In [5]:
train_dict = train_df[:500].to_dict(orient='records')
val_dict = val_df[:250].to_dict(orient='records')

In [6]:
train_dict[0]

{'id': 'isdt_tut-689',
 'sent': "Il diritto dell'usufruttuario non si estende al tesoro che si scopra durante l'usufrutto, salve le ragioni che gli possono competere come ritrovatore (932).",
 'category': 8,
 'n_tokens': 31,
 'char_per_tok': 4.81481481481482,
 'upos_dist_DET': 16.1290322580645,
 'upos_dist_ADV': 3.2258064516129,
 'upos_dist_PUNCT': 12.9032258064516,
 'upos_dist_NUM': 3.2258064516129,
 'upos_dist_PRON': 16.1290322580645,
 'upos_dist_ADP': 12.9032258064516,
 'upos_dist_PROPN': 0.0,
 'upos_dist_ADJ': 3.2258064516129,
 'upos_dist_VERB': 9.67741935483871,
 'upos_dist_NOUN': 19.3548387096774,
 'upos_dist_CCONJ': 0.0,
 'upos_dist_AUX': 3.2258064516129,
 'avg_links_len': 2.42307692307692,
 'max_links_len': 11,
 'avg_max_depth': 5,
 'dep_dist_obj': 0.0,
 'dep_dist_nsubj': 12.9032258064516,
 'subj_pre': 75.0,
 'subj_post': 25.0,
 'n_prepositional_chains': 1,
 'avg_prepositional_chain_len': 1.0,
 'avg_subordinate_chain_len': 1.0,
 'subordinate_proposition_dist': 66.6666666666667,

In [8]:
#funzione per ottenere gli embedding delle frasi.
def feature_extraction(samples, model_name):
    first_layer = 1
    last_layer = 8
    tokenizer = BertTokenizer.from_pretrained("dbmdz/bert-base-italian-cased") #come tokenizzatore utilizziamo il solito giusto (bert-base-italian-uncased)? 
    model = BertForMaskedLM.from_pretrained(model_name)
    for sample in tqdm(samples, desc="Estrazione features", unit="sample"):
        encoded_sen = tokenizer(sample["sent"], padding=True, truncation=True, max_length=128, return_tensors='pt') 
        with torch.no_grad():    
            model_output = model(**encoded_sen, output_hidden_states=True)
            hidden_states = model_output.hidden_states
            for layer in range(first_layer, last_layer+1):
                layer_output = torch.squeeze(hidden_states[layer])
                cls_embedding = layer_output[0, :].cpu().detach().numpy()
                sample[f'layer_{layer}'] = {'cls_embedding': cls_embedding}
    return samples

#funzione per ottenere features e lables
def get_features_lables(samples, feature, layer):
    X = []
    y = []
    for sample in samples:
        X = sample[layer]["cls_embedding"]
        y = sample[feature]
    return X, y

#funzione per addestrare e valutare il modello
def train_eval(train_set, val_set, feature, layer):
    scaler = MinMaxScaler()
    X_train, y_train = get_features_lables(train_set, feature, layer)
    X_val, y_val = get_features_lables(val_set, feature, layer)
    scaled_X_train = scaler.fit_transform(X_train)
    scaled_X_val = scaler.transform(X_val)
    clf = sklearn.linear_model.Ridge(alpha=1.0)
    clf.fit(scaled_X_train, y_train)
    y_pred = clf.predict(scaled_X_val)
    return classification_report(y_val, y_pred, output_dict=True) 

In [14]:
final_model_path =  f"C:/Users/bergo/OneDrive - University of Pisa/Tesi Magistrale/models/ANTI_CURRICULUM/final_pretrained_model"
samples = feature_extraction(val_dict, final_model_path)

Estrazione features: 100%|██████████| 250/250 [00:09<00:00, 26.22sample/s]


In [9]:
# DEVO ITERARE ANCHE SUI CHECKPOINT CHE USIAMO
checkpoints = [2, 32, 512, 8192, 0] 
ling_features = ["n_tokens", "upos_dist_ADP", "char_per_tok"]

In [10]:
def probing_checkpoints(checkpoints, training_id, train_dict, val_dict, ling_features):
    first_layer = 1
    last_layer = 8
    results = dict()
    for n_step in checkpoints:
        checkpoint_name = f'checkpoint-step{n_step}'
        if n_step == 0:
            model_name = f"C:/Users/bergo/OneDrive - University of Pisa/Tesi Magistrale/models/{training_id}/final_pretrained_model"
            print("Inizio probing per il modello finale")
        else:
            model_name = f"C:/Users/bergo/OneDrive - University of Pisa/Tesi Magistrale/models/{training_id}/checkpoints/{checkpoint_name}"
            print(f"Inizio probing per il checkpoint {n_step}")
        
        print("Estrazione delle feature di training...")
        train_samples = feature_extraction(train_dict, model_name)  #si effettua l'estrazione delle feature per il checkpoint
        print("Estrazione delle features di validation...")
        val_samples = feature_extraction(val_dict, model_name)   
        for ling_feature in ling_features:              
            print(f'Addestramento del modello sulla feature linguistica: {ling_feature}') 
            layer_results = dict()
            feature_result = dict()
            for layer in range(first_layer, last_layer+1):
                print(f"Training for layer {layer}/{last_layer}")
                layer_result = train_eval(train_samples, val_samples, ling_feature, f'layer_{layer}')   #addestriamo il ridge per ogni layer ottenendo le metriche
                layer_results[f'layer_{layer}'] = layer_result
            feature_result[ling_feature] = {"results": layer_results}
            results[f"checkpoint{n_step}"] = feature_result
    return results
        

In [11]:
results = probing_checkpoints(checkpoints, "ANTI_CURRICULUM", train_dict, val_dict, ling_features)

Inizio probing per il checkpoint 2
Estrazione delle feature di training...


Estrazione features: 100%|██████████| 500/500 [00:21<00:00, 22.81sample/s]


Estrazione delle features di validation...


Estrazione features: 100%|██████████| 250/250 [00:11<00:00, 21.55sample/s]


Addestramento del modello sulla feature linguistica: n_tokens
Training for layer 1/8


ValueError: Expected 2D array, got 1D array instead:
array=[-2.1315079   2.5852509   0.8696965  -0.05993515  1.0990232   0.25199714
  1.0477796   0.6888469  -0.3031923   0.57987154  1.1484693   0.01575219
 -0.19587785  0.8250259  -0.28469267  1.1389377   0.2756512   1.8663802
 -0.53986233  0.6415098   0.34137812 -0.3464678   0.12808228 -1.4836715
  2.1205585   0.01961643  1.8258386   0.17781517 -0.6131355   0.7713341
  0.7724039  -0.4233626  -0.46201268  0.31877062  1.3882495   0.0500122
 -0.5601297  -0.31841275 -0.9053752   0.16928726 -1.3684188  -1.2948371
 -1.0609267   0.9092711   0.69348264 -0.02238305  0.27656123 -1.9663436
  1.1278688  -0.19321296  0.6869232  -1.4053665   0.6567181  -0.1321012
  0.2503742  -0.41009977 -0.6352198   1.3595886  -1.0228201  -0.1098899
  1.5206287  -0.24586315  0.5683745   1.3556516  -1.4159044   0.6217237
  0.6803576   0.17879528 -1.2983041  -0.32037452  0.61688304 -0.03982257
  0.2194626  -0.86114943 -1.3924428   0.54315287  1.1556351  -0.5546677
 -

In [ ]:
import json

In [ ]:
with open("C:/Users/bergo/OneDrive - University of Pisa/Tesi Magistrale/models/ANTI_CURRICULUM/probing.txt", "w") as f:
    json.dump(results, f)